<a href="https://colab.research.google.com/github/zhang-linnng/convex_restriction_transformed/blob/main/(current)0912_mincosts_case123nick.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!apt-get install -y -qq glpk-utils
!apt-get install -y -qq libglpk-dev
!pip install cvxpy

In [3]:
!pip install pypower

In [4]:
%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

In [5]:
!ipopt --version

Ipopt 3.13.2 (x86_64-pc-linux-gnu), ASL(20190605)



In [6]:
!which ipopt

bin/ipopt


# Import libraries

In [7]:
from __future__ import division
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import csv
from mpl_toolkits.mplot3d import Axes3D
import random
import pandas as pd
import math
import json
import pickle
import cmath
import sys

random.seed(1234321)

# Project folder

In [8]:
def create_dir(path):
    isExist = os.path.exists(path)

    if not isExist:
        # Create a new directory if it does not exist
        os.makedirs(path)
        print("The new directory %s is created!" % (path))

In [9]:
project_path = "/content/gdrive/MyDrive/2023Projects/convex_restriction_projection/"
grid_path = project_path + "grid_case123nick/"

subproject1 = 'minimize_losses/'
subproject2 = 'minimize_cost/'
subproject3 = 'state_estimation/'
our_method_path = 'proposed/'
socp_path = 'socp/'

In [10]:
create_dir(project_path)

for project in [subproject1, subproject2, subproject3]:
    for method in [our_method_path, socp_path]:
        create_dir(project_path+project+method)

# Grid

In [11]:
import scipy.io as sio

# mat_contents = sio.loadmat(settings_path + mat_fname)
branch_contents = sio.loadmat(grid_path + 'branch_case123nick.mat')
bus_contents = sio.loadmat(grid_path + 'bus_case123nick.mat')
gen_contents = sio.loadmat(grid_path + 'gen_case123nick.mat')

print(branch_contents.keys())
print(bus_contents.keys())
print(gen_contents.keys())

dict_keys(['__header__', '__version__', '__globals__', 'branch'])
dict_keys(['__header__', '__version__', '__globals__', 'bus'])
dict_keys(['__header__', '__version__', '__globals__', 'gen'])


In [12]:
## branch data: fbus(0), tbus(1), r(2), x(3), b(4)
## bus data: bus(0), type(1), Pd(2), Qd(3), Vmx(11), Vmn(12)
## gen data: bus(0), Pg(1), Qg(2), Qmax(3), Qmin(4), Pmax(8), Pmin(9)

branch_data = branch_contents['branch']
bus_data = bus_contents['bus']
gen_data = gen_contents['gen']

branch_array = np.concatenate([branch_data[:,0:1], branch_data[:,1:2], branch_data[:,2:3], branch_data[:,3:4], branch_data[:,4:5]], axis=1)
branch_df = pd.DataFrame(branch_array, columns = ['fbus','tbus','r','x','b'])
branch_df['fbus'] = branch_df['fbus'].apply(lambda x: int(x))
branch_df['tbus'] = branch_df['tbus'].apply(lambda x: int(x))

bus_array = np.concatenate([bus_data[:,0:1], bus_data[:,1:2], bus_data[:,2:3], bus_data[:,3:4], bus_data[:,11:12], bus_data[:,12:13]], axis=1)
bus_df = pd.DataFrame(bus_array, columns = ['bus','type','Pd','Qd','Vmx','Vmn'])
bus_df['bus'] = bus_df['bus'].apply(lambda x: int(x))
bus_df['type'] = bus_df['type'].apply(lambda x: int(x))

gen_array = np.concatenate([gen_data[:,0:1], gen_data[:,1:2], gen_data[:,2:3], gen_data[:,3:4], gen_data[:,4:5], gen_data[:,8:9], gen_data[:,9:10]], axis=1)
gen_df = pd.DataFrame(gen_array, columns = ['bus','Pg','Qg','Qmax','Qmin','Pmax','Pmin'])
gen_df['bus'] = gen_df['bus'].apply(lambda x: int(x))

In [13]:
for m in range(branch_df.shape[0]):
    f_end = branch_df.iloc[m, 0]
    t_end = branch_df.iloc[m, 1]
    if f_end > t_end:
        print('f_bus {}, t_bus {}'.format(f_end, t_end))

f_bus 14, t_bus 11
f_bus 14, t_bus 10
f_bus 34, t_bus 15
f_bus 113, t_bus 61
f_bus 114, t_bus 1


In [14]:
branch_df_copy = branch_df.copy()
branch_df_copy = branch_df_copy.sort_values(by=['fbus', 'tbus'], ascending=True)

for m in range(branch_df_copy.shape[0]):
    f_end = branch_df_copy.iloc[m, 0]
    t_end = branch_df_copy.iloc[m, 1]
    if f_end > t_end:
        print('f_bus {}, t_bus {}'.format(f_end, t_end))

f_bus 14, t_bus 10
f_bus 14, t_bus 11
f_bus 34, t_bus 15
f_bus 113, t_bus 61
f_bus 114, t_bus 1


In [15]:
branch_df_copy = branch_df.copy()
branch_df_copy = branch_df_copy.sort_values(by=['fbus', 'tbus'], ascending=True)

r_ij = branch_df_copy["r"].to_numpy()
x_ij = branch_df_copy["x"].to_numpy()
g_ij = r_ij/(r_ij**2+x_ij**2)
b_ij = x_ij/(r_ij**2+x_ij**2)
branch_df_copy["conductance(g)"] = g_ij
branch_df_copy["susceptance(b)"] = b_ij
branch_df_copy.head()

,fbus,tbus,r,x,b,conductance(g),susceptance(b)
0,1,2,0.002546,0.002581,0.000003,193.723868,196.391040
1,1,3,0.003637,0.003687,0.000004,135.606729,137.473712
2,1,7,0.001502,0.003539,0.000006,101.623767,239.402135
3,3,4,0.002909,0.002949,0.000003,169.508376,171.842151
4,3,5,0.004728,0.004793,0.000005,104.312855,105.749003


In [16]:
count_lines_df = branch_df_copy.groupby(['fbus', 'tbus']).count()
count_lines_df.head()
branches = count_lines_df.index.to_list()
branches_list = [(int(x[0]), int(x[1])) for x in branches]
print(branches_list)

nodes = bus_df['bus'].to_list()
nodes_list = [int(x) for x in nodes]
print(nodes_list)

num_nodes = len(nodes_list)
num_branches = len(branches_list)
print('num_nodes:', num_nodes)
print('num_branches:', num_branches)

[(1, 2), (1, 3), (1, 7), (3, 4), (3, 5), (5, 6), (7, 8), (8, 9), (8, 12), (8, 13), (9, 14), (13, 18), (13, 34), (13, 52), (14, 10), (14, 11), (15, 16), (15, 17), (18, 19), (18, 21), (18, 35), (19, 20), (21, 22), (21, 23), (23, 24), (23, 25), (25, 26), (25, 28), (26, 27), (26, 31), (27, 33), (28, 29), (29, 30), (31, 32), (34, 15), (35, 36), (35, 40), (36, 37), (36, 38), (38, 39), (40, 41), (40, 42), (42, 43), (42, 44), (44, 45), (44, 47), (45, 46), (47, 48), (47, 49), (49, 50), (50, 51), (52, 53), (53, 54), (54, 55), (54, 57), (55, 56), (57, 58), (57, 60), (58, 59), (60, 62), (60, 67), (62, 63), (63, 64), (64, 65), (65, 66), (67, 68), (67, 72), (67, 97), (68, 69), (69, 70), (70, 71), (72, 73), (72, 76), (73, 74), (74, 75), (76, 77), (76, 86), (77, 78), (78, 79), (78, 80), (80, 81), (81, 82), (81, 84), (82, 83), (84, 85), (86, 87), (87, 88), (87, 89), (89, 90), (89, 91), (91, 92), (91, 93), (93, 94), (93, 95), (95, 96), (97, 98), (97, 101), (98, 99), (99, 100), (101, 102), (101, 105), (1

In [17]:
gen_set_mask = bus_df['type']>1
gen_set = bus_df['bus'][gen_set_mask]
gen_rows = np.array(range(num_nodes))[gen_set_mask]
print('gen_set:\n', gen_set)
print('gen_rows:\n', gen_rows)

bus_lists = bus_df['bus'].to_numpy()
print(bus_lists.shape)
print(bus_lists)

branch_df_copy['index'] = branch_df_copy.index
branch_df_copy.head()

b_for_lines = b_ij.reshape(-1,1)
g_for_lines = g_ij.reshape(-1,1)

gen_set:
 113    114
Name: bus, dtype: int64
gen_rows:
 [113]
(114,)
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114]


In [18]:
numeric_to_bus = dict()
for idx, bus_name in enumerate(bus_df['bus'].tolist()):

    numeric_to_bus[idx] = bus_name
print('numeric_to_bus:', numeric_to_bus)

bus_to_numeric = dict()
for k in range(num_nodes):
    node_idx = nodes_list[k]
    bus_to_numeric[node_idx] = k

bus_to_neighbors = dict()
for edge in branches_list:
    fbus = edge[0]
    tbus = edge[1]
    if fbus not in bus_to_neighbors.keys():
        bus_to_neighbors[fbus] = [tbus]
    else:
        bus_to_neighbors[fbus] += [tbus]

    if tbus not in bus_to_neighbors.keys():
        bus_to_neighbors[tbus] = [fbus]
    else:
        bus_to_neighbors[tbus] += [fbus]

print('bus_to_numeric:', bus_to_numeric)
print('bus_to_neighbors:', bus_to_neighbors)

numbering_links = dict()
for i in range(num_branches):
    fbus = branch_df_copy.iloc[i, 0]
    tbus = branch_df_copy.iloc[i, 1]
    numbering_links[i] = (fbus, tbus)
print('numbering_links:', numbering_links)

bus_types = []
for i in range(num_nodes):
    match bus_df.iloc[i,1]:
        case 1:
            # print('PQ bus')
            bus_types.append('load')
        case 2:
            # print('PV bus')
            bus_types.append('gen')
        case 3:
            # print('reference bus')
            bus_types.append('slack')
        case _:
            # print('Isolated bus')
            bus_types.append('isolated')

print('bus types:', bus_types)

numeric_to_bus: {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 9: 10, 10: 11, 11: 12, 12: 13, 13: 14, 14: 15, 15: 16, 16: 17, 17: 18, 18: 19, 19: 20, 20: 21, 21: 22, 22: 23, 23: 24, 24: 25, 25: 26, 26: 27, 27: 28, 28: 29, 29: 30, 30: 31, 31: 32, 32: 33, 33: 34, 34: 35, 35: 36, 36: 37, 37: 38, 38: 39, 39: 40, 40: 41, 41: 42, 42: 43, 43: 44, 44: 45, 45: 46, 46: 47, 47: 48, 48: 49, 49: 50, 50: 51, 51: 52, 52: 53, 53: 54, 54: 55, 55: 56, 56: 57, 57: 58, 58: 59, 59: 60, 60: 61, 61: 62, 62: 63, 63: 64, 64: 65, 65: 66, 66: 67, 67: 68, 68: 69, 69: 70, 70: 71, 71: 72, 72: 73, 73: 74, 74: 75, 75: 76, 76: 77, 77: 78, 78: 79, 79: 80, 80: 81, 81: 82, 82: 83, 83: 84, 84: 85, 85: 86, 86: 87, 87: 88, 88: 89, 89: 90, 90: 91, 91: 92, 92: 93, 93: 94, 94: 95, 95: 96, 96: 97, 97: 98, 98: 99, 99: 100, 100: 101, 101: 102, 102: 103, 103: 104, 104: 105, 105: 106, 106: 107, 107: 108, 108: 109, 109: 110, 110: 111, 111: 112, 112: 113, 113: 114}
bus_to_numeric: {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8:

## Incidence matrix

In [19]:
# incidence matrix
E = np.zeros((num_branches, num_nodes))
for m in range(num_branches):
    f_end = branch_df_copy.iloc[m, 0]
    t_end = branch_df_copy.iloc[m, 1]

    col1 = bus_to_numeric[f_end]
    col2 = bus_to_numeric[t_end]

    if col1 < col2:
        E[m, col1] = 1
        E[m, col2] = -1
    else:
        E[m, col1] = -1
        E[m, col2] = 1

print('E:', E)

E: [[ 1. -1.  0. ...  0.  0.  0.]
 [ 1.  0. -1. ...  0.  0.  0.]
 [ 1.  0.  0. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  1. -1.  0.]
 [ 0.  0.  0. ...  0. -1.  0.]
 [ 1.  0.  0. ...  0.  0. -1.]]


#  Vectorize power flow equations

In [20]:
G_a = np.zeros((num_nodes, num_branches))
for row in range(num_nodes):
    for col in range(num_branches):
        row_bus = bus_df['bus'].iloc[row]
        end1 = branch_df_copy.iloc[col, 0]
        end2 = branch_df_copy.iloc[col, 1]
        edge = (end1, end2)

        if row_bus not in edge:
            G_a[row, col] = 0

        else: # if bus in edge
            if edge[0] == row_bus:
                neighbor = edge[1]
            else:
                neighbor = edge[0]

            # convert bus no. to numerical indexes
            neighbor_idx = bus_to_numeric[neighbor]


            if row > neighbor_idx: # if row > neighbor
                G_a[row, col] = -g_ij[col]

            else: # if row < neighbor
                G_a[row, col] = -g_ij[col]


B_a = np.zeros((num_nodes, num_branches))
for row in range(num_nodes):
    for col in range(num_branches):
        row_bus = bus_df['bus'].iloc[row]
        end1 = branch_df_copy.iloc[col, 0]
        end2 = branch_df_copy.iloc[col, 1]
        edge = (end1, end2)

        if row_bus not in edge:
            B_a[row, col] = 0

        else: # if bus in edge
            if edge[0] == row_bus:
                neighbor = edge[1]
            else:
                neighbor = edge[0]

            # convert bus no. to numerical indexes
            neighbor_idx = bus_to_numeric[neighbor]

            if row > neighbor_idx: # if row > neighbor
                B_a[row, col] = -b_ij[col]

            else: # if row < neighbor
                B_a[row, col] = b_ij[col]


print('G_a:', G_a.shape)
print('B_a:', B_a.shape)

G_hat = np.zeros((num_nodes, num_nodes))
for m in range(num_branches):
    end1 = branch_df_copy.iloc[m, 0]
    end2 = branch_df_copy.iloc[m, 1]

    row = bus_to_numeric[end1]
    col = bus_to_numeric[end2]

    G_hat[row, col] = g_ij[m]
    G_hat[col, row] = g_ij[m]

print('G_hat:', G_hat.shape)


d_active = np.zeros((num_nodes, 1))
for row in range(num_nodes):

    d_active[row, 0] = np.sum(G_hat[row,:])

print('d_active:', d_active.shape)

A = np.block([
    G_a, B_a, d_active
])
print('A:', A.shape)

G_a: (114, 113)
B_a: (114, 113)
G_hat: (114, 114)
d_active: (114, 1)
A: (114, 227)


In [21]:
G_r = np.zeros((num_nodes, num_branches))
for row in range(num_nodes):
    for col in range(num_branches):
        row_bus = bus_df['bus'].iloc[row]
        end1 = branch_df_copy.iloc[col, 0]
        end2 = branch_df_copy.iloc[col, 1]
        edge = (end1, end2)

        if row_bus not in edge:
            G_r[row, col] = 0

        else: # if bus in edge
            if edge[0] == row_bus:
                neighbor = edge[1]
            else:
                neighbor = edge[0]

            # convert bus no. to numerical indexes
            neighbor_idx = bus_to_numeric[neighbor]


            if row > neighbor_idx: # if row > neighbor
                G_r[row, col] = g_ij[col]

            else: # if row < neighbor
                G_r[row, col] = -g_ij[col]


B_r = np.zeros((num_nodes, num_branches))
for row in range(num_nodes):
    for col in range(num_branches):
        row_bus = bus_df['bus'].iloc[row]
        end1 = branch_df_copy.iloc[col, 0]
        end2 = branch_df_copy.iloc[col, 1]
        edge = (end1, end2)

        if row_bus not in edge:
            B_r[row, col] = 0

        else: # if bus in edge
            if edge[0] == row_bus:
                neighbor = edge[1]
            else:
                neighbor = edge[0]

            # convert bus no. to numerical indexes
            neighbor_idx = bus_to_numeric[neighbor]

            if row > neighbor_idx: # if row > neighbor
                B_r[row, col] = -b_ij[col]

            else: # if row < neighbor
                B_r[row, col] = -b_ij[col]


print('G_r:', G_r.shape)
print('B_r:', B_r.shape)


B_hat = np.zeros((num_nodes, num_nodes))
for m in range(num_branches):
    end1 = branch_df_copy.iloc[m, 0]
    end2 = branch_df_copy.iloc[m, 1]

    row = bus_to_numeric[end1]
    col = bus_to_numeric[end2]

    B_hat[row, col] = b_ij[m]
    B_hat[col, row] = b_ij[m]

print('B_hat:', B_hat.shape)

d_reactive = np.zeros((num_nodes, 1))
for row in range(num_nodes):

    d_reactive[row, 0] = np.sum(B_hat[row,:])

print('d_reactive:', d_reactive.shape)

R = np.block([
    B_r, G_r, d_reactive
])
print('R:', R.shape)

G_r: (114, 113)
B_r: (114, 113)
B_hat: (114, 114)
d_reactive: (114, 1)
R: (114, 227)


## P, Q as functions of phase angle

In [22]:
def p_in_voltage(theta):

    x = theta.reshape(-1,1)
    z = np.sin(E@x)

    return G_a @ np.sqrt(1-z**2) + B_a @ z + d_active


def q_in_voltage(theta):

    x = theta.reshape(-1,1)
    z = np.sin(E@x)

    return B_r @ np.sqrt(1-z**2) + G_r @ z + d_reactive

nominal_angle = np.zeros(num_nodes)
p = p_in_voltage(nominal_angle)
q = q_in_voltage(nominal_angle)
# print(p)
# print(q)

## P, Q as functions of z

In [23]:
import cvxpy as cp

In [24]:
def cp_p_in_transformed_space(z):

    return G_a @ cp.sqrt(1-cp.square(z)) + B_a @ z + d_active

def cp_q_in_transformed_space(z):

    return B_r @ cp.sqrt(1-cp.square(z)) + G_r @ z + d_reactive


def p_in_transformed_space(z):

    z = z.reshape(-1, 1)

    return G_a @ np.sqrt(1-z**2) + B_a @ z + d_active


def q_in_transformed_space(z):

    z = z.reshape(-1, 1)

    return B_r @ np.sqrt(1-z**2) + G_r @ z + d_reactive

# def recover_phase_angle(z):
#     z = np.squeeze(z)

#     all_edges = branch_df[['fbus', 'tbus']].values
#     recovered = dict()
#     for row in bus_df['bus'].values:
#         recovered[row] = 0.

#     for i, row in enumerate(all_edges):
#         print("The edge is: {}, and the angle difference is: {}".format(row, np.arcsin(z[i])))
#         node1 = row[0]
#         node2 = row[1]
#         if node1 < node2:
#             recovered[node2] = recovered[node1] - np.arcsin(z[i])
#         else:
#             recovered[node1] = recovered[node2] - np.arcsin(z[i])

#     x = list(recovered.values())
#     print('recovered values:', recovered)

#     return recovered

## Check solution feasibility

In [25]:
# check if the given solution is a feasible one
def is_phase_angle_feasible(theta, bus_dataframe):

    Qmax = bus_dataframe['Qmax'].to_numpy()
    Qmin = bus_dataframe['Qmin'].to_numpy()
    Pmax = bus_dataframe['Pmax'].to_numpy()
    Pmin = bus_dataframe['Pmin'].to_numpy()

    p = p_in_voltage(theta)
    q = q_in_voltage(theta)

    # print('p_max:\n', np.squeeze(Pmax))
    # print('p:\n', np.squeeze(p))
    # print('p_min:\n', np.squeeze(Pmin))

    # print('q_max:\n', np.squeeze(Qmax))
    # print('q:\n', np.squeeze(q))
    # print('q_min:\n', np.squeeze(Qmin))

    p_values = np.squeeze(p)
    q_values = np.squeeze(q)

    tolerance = -1e-6

    # if np.all(p_values>=Pmin) & np.all(p_values<=Pmax) & np.all(q_values >=Qmin) & np.all(q_values<=Qmax):
    if np.all(p_values-Pmin>=tolerance) & np.all(Pmax-p_values>=tolerance) & np.all(q_values-Qmin>=tolerance) & np.all(Qmax-q_values>=tolerance):

        return True

    else:

        return False


# nominal_angle = np.zeros(num_nodes)
# is_voltage_feasible(nominal_angle) # Nooope

def is_z_feasible(z, bus_dataframe):

    Qmax = bus_dataframe['Qmax'].to_numpy()
    Qmin = bus_dataframe['Qmin'].to_numpy()
    Pmax = bus_dataframe['Pmax'].to_numpy()
    Pmin = bus_dataframe['Pmin'].to_numpy()

    p = p_in_transformed_space(z)
    q = q_in_transformed_space(z)

    p_values = np.squeeze(p)
    q_values = np.squeeze(q)

    # print('p_max:\n', np.squeeze(Pmax))
    # print('p:\n', np.squeeze(p))
    # print('p_min:\n', np.squeeze(Pmin))

    # print('q_max:\n', np.squeeze(Qmax))
    # print('q:\n', np.squeeze(q))
    # print('q_min:\n', np.squeeze(Qmin))
    tolerance = -1e-6

    print('violate Pmin?',np.all(p_values-Pmin>=tolerance))
    print('violate Pmax?',np.all(Pmax-p_values>=tolerance))
    print('violate Qmin?',np.all(q_values-Qmin>=tolerance))
    print('violate Qmax?',np.all(Qmax-q_values>=tolerance))

    # if np.all(p_values>=Pmin) & np.all(p_values<=Pmax) & np.all(q_values >=Qmin) & np.all(q_values<=Qmax):
    if np.all(p_values-Pmin>=tolerance) & np.all(Pmax-p_values>=tolerance) & np.all(q_values-Qmin>=tolerance) & np.all(Qmax-q_values>=tolerance):

        return True

    else:

        return False


# nominal_z = np.zeros(num_branches)
# is_z_feasible(nominal_z)


# Bounds

## Original bounds

In [26]:
def set_bounds():

    added_bus_df = bus_df.copy()
    gen_df_copy = gen_df.copy()
    gen_df_copy = gen_df_copy.set_index('bus')

    added_bus_df['Pd'] = added_bus_df['Pd'] * 10
    added_bus_df['Qd'] = added_bus_df['Qd'] * 10

    for i in range(num_nodes):
        bus = added_bus_df.iloc[i, 0]
        type = added_bus_df.iloc[i, 1]
        if type > 1: # generator or slack(reference) buses, copy everything
            added_bus_df.loc[i, 'Pmax']= gen_df_copy.loc[bus, 'Pmax']
            added_bus_df.loc[i, 'Pmin']= 0
            added_bus_df.loc[i, 'Qmax']= gen_df_copy.loc[bus, 'Qmax']
            added_bus_df.loc[i, 'Qmin']= gen_df_copy.loc[bus, 'Qmin']

        else:
        # load buses, Pmax = -Pd; Pmin

            added_bus_df.loc[i, 'Pmax'] = -added_bus_df['Pd'][i]
            added_bus_df.loc[i, 'Pmin'] = -10. # set a maximum loading at this bus,
            # based on the generators's capacity, which is 10, I set the maximum loading to be 10

            # # this results in infeasibility
            # bus_df.loc[i, 'Qmax'] = bus_df['Qd'][i] # positive Qmax means maximum reactive power that
            # # this load bus can consume
            # bus_df.loc[i, 'Qmin'] = 0. # postive Q min means that this load bus cannot contribute(generate)
            # # reactive power
            added_bus_df.loc[i, 'Qmax'] = 10. # this is the maximum reactive power that generator bus can contribute
            added_bus_df.loc[i, 'Qmin'] = bus_df['Qd'][i]

    print('total (active) demand is {}'.format(added_bus_df['Pd'].sum()))
    print('total (reactive) demand is {}'.format(added_bus_df['Qd'].sum()))
    print('maximum generator active contribution is {}'.format(np.maximum(added_bus_df['Pmax'],0.0).sum()))
    print('maximum generator reactive contribution is {}'.format(np.minimum(added_bus_df['Qmin'],0.0).sum()))


    return added_bus_df

# added_bus_df = set_bounds()
# added_bus_df.head()

## Modify bounds

In [27]:
def modify_bounds(original_bus_df):
    original_df_copy = original_bus_df.copy()

    # generate phase angle difference randomly
    initial_guess = sio.loadmat(project_path + 'case123nick_voltage_angles.mat')

    dc_va_degrees = initial_guess['voltage_angles']
    # print('dc_va shape:', dc_va_degrees.shape)
    # print('dc_va:', np.squeeze(dc_va_degrees)[:10])

    dc_va_radians = dc_va_degrees / 180 * np.pi
    transfom_to_z =  np.sin(E @ dc_va_radians)
    # print('transfom_to_z:', np.squeeze(transfom_to_z)[:10])

    candidate_z_values = np.squeeze(transfom_to_z).tolist()
    candidate_z_values = [np.round(x, 4) for x in candidate_z_values]
    # print('candidate_z_values:', candidate_z_values[:10])

    z_values = np.array(candidate_z_values) * 4
    print('is it feasible?', is_z_feasible(z_values, original_df_copy))
    print('z_values:', z_values[:10])

    nominal_p = p_in_transformed_space(z_values)
    nominal_q = q_in_transformed_space(z_values)

    Qmax = original_df_copy['Qmax'].to_numpy().reshape(-1, 1)
    Qmin = original_df_copy['Qmin'].to_numpy().reshape(-1, 1)
    Pmax = original_df_copy['Pmax'].to_numpy().reshape(-1, 1)
    Pmin = original_df_copy['Pmin'].to_numpy().reshape(-1, 1)
    nominal_p = np.array(nominal_p).reshape(-1, 1)
    nominal_q = np.array(nominal_q).reshape(-1, 1)

    results = np.concatenate([Pmax, nominal_p, Pmin, Qmax, nominal_q, Qmin], axis=1)
    column_names = ['p_max', 'nominal_p', 'p_min', 'q_max', 'nominal_q', 'q_min']
    results_df = pd.DataFrame(results, columns=column_names)
    print(results_df)

    new_Pmax = np.maximum(np.squeeze(Pmax), np.squeeze(nominal_p))
    new_Pmin = np.minimum(np.squeeze(Pmin), np.squeeze(nominal_p))

    new_Qmax = np.maximum(np.squeeze(Qmax), np.squeeze(nominal_q))
    new_Qmin = np.minimum(np.squeeze(Qmin), np.squeeze(nominal_q))

    counts = 0
    for i in range(num_nodes):
        if nominal_p[i] > 0.0:# a generator
            new_Pmin[i] = 0.0
            print('Qmax:', new_Qmax[i])
            print('Qmin:', new_Qmin[i])
            counts+=1
    print('counts of gens:', counts)

    original_df_copy2 = original_bus_df.copy()
    original_df_copy2['Pmax'] = new_Pmax
    original_df_copy2['Pmin'] = new_Pmin
    original_df_copy2['Qmax'] = new_Qmax
    original_df_copy2['Qmin'] = new_Qmin
    print(original_df_copy2.tail())

    print('is it feasible?', is_z_feasible(z_values, original_df_copy2))

    original_df_copy2.to_json(project_path + 'mod_bus_dataframe.json')
    np.save(project_path + 'nominal_z_values.npy', z_values)


    return

# modify_bounds(added_bus_df)

# Generate cost coefficients

In [28]:
def generate_costs():
    df = pd.DataFrame(pd.read_json(project_path + 'mod_bus_dataframe.json'))

    Qmax = df['Qmax'].to_numpy()
    Qmin = df['Qmin'].to_numpy()
    Pmax = df['Pmax'].to_numpy()
    Pmin = df['Pmin'].to_numpy()

    # random.seed(138206)
    random.seed(12306)

    cost_coefficients_list = []
    count_distributed_gen = 0
    distributed_gen_name_list = []
    distributed_gen_idx_list = []

    for i in range(num_nodes):
        Pi_max = Pmax[i]
        # Qi_min = Qmin[i]
        if Pi_max <= 0: # a load
            random_cost = np.round(-np.random.randint(1, num_nodes)/num_nodes, 2)*np.random.randint(1, 10)
            cost_coefficients_list.append(random_cost)
        else: # a generator
            random_cost = np.round(np.random.randint(1, num_nodes)/num_nodes, 2)*np.random.randint(1, 10)
            cost_coefficients_list.append(random_cost)

            count_distributed_gen+=1
            distributed_gen_name_list.append(bus_df.loc[i,'bus'])
            distributed_gen_idx_list.append(i)

    print('cost_coefficients_list:', cost_coefficients_list[-10:])
    print('counts:', count_distributed_gen)
    print('distributed_gen_name_list:', distributed_gen_name_list)
    print('distributed_gen_idx_list:', distributed_gen_idx_list)

    distributed_gencost_data = dict()
    distributed_gencost_data['cost_coefficients'] = cost_coefficients_list
    distributed_gencost_data['distributed_gen_name_list'] = distributed_gen_name_list
    distributed_gencost_data['distributed_gen_idx_list'] = distributed_gen_idx_list

    # Save the dictionary to the pickle file
    with open(project_path+subproject2 + 'distributed_gencost_data.pkl', 'wb') as file:
        pickle.dump(distributed_gencost_data, file)

# generate_costs()

# Proposed method

## Input a strictly feasible (interior) point

In [29]:
mod_bus_df = pd.DataFrame(pd.read_json(project_path + 'mod_bus_dataframe.json'))

Qmax = mod_bus_df['Qmax'].to_numpy()
Qmin = mod_bus_df['Qmin'].to_numpy()
Pmax = mod_bus_df['Pmax'].to_numpy()
Pmin = mod_bus_df['Pmin'].to_numpy()

In [57]:
ITER_NUM = 3

In [58]:
if ITER_NUM == 0:
    z_interior = np.load(project_path + 'nominal_z_values.npy')
    print('is it feasible?', is_z_feasible(z_interior, mod_bus_df))
else:
    z_interior = np.load(project_path+subproject2+'iter' + str(ITER_NUM) +'_z_interior.npy')

## Linearize P, Q

In [48]:
def f_prime(x):

    return -x/np.sqrt(1-x**2)


def Jacobian(z):
    z = np.squeeze(z)

    return np.diag(f_prime(z))


def linearize_Pi(bus_i, z_base, z):
    z_base = z_base.reshape(-1,1)

    G_a_rowi = G_a[bus_i,:].reshape(1,-1)
    B_a_rowi = B_a[bus_i,:].reshape(1,-1)

    Pi_z_base = G_a_rowi @ np.sqrt(1-z_base**2) + B_a_rowi @ z_base + d_active[bus_i,0]
    Pi_z_base = Pi_z_base[0,0]

    dp_dz_base = G_a_rowi @ Jacobian(z_base) + B_a_rowi # a row vector
    # print('dp_dz_base:', dp_dz_base.shape)
    linearization = Pi_z_base + dp_dz_base @ (z-z_base)
    # print('linearization:', linearization.shape)

    return linearization[0,0]


def linearize_Qi(bus_i, z_base, z):
    z_base = z_base.reshape(-1,1)

    B_r_rowi = B_r[bus_i,:].reshape(1,-1)
    G_r_rowi = G_r[bus_i,:].reshape(1,-1)

    Qi_z_base = B_r_rowi @ np.sqrt(1-z_base**2) + G_r_rowi @ z_base + d_reactive[bus_i,0]
    Qi_z_base = Qi_z_base[0,0]

    dq_dz_base = B_r_rowi @ Jacobian(z_base) + G_r_rowi # a row vector
    # print('dp_dz_base:', dp_dz_base.shape)
    linearization = Qi_z_base + dq_dz_base @ (z-z_base)
    # print('linearization:', linearization.shape)

    return linearization[0,0]


## Find base points for each lower bound constraint

### Find base points for Pi_min

In [33]:
def basePoint_of_Pi(bus_name, z_nominal):

    print('Linearize P{}...'.format(bus_name))
    # print('An interior point is ', np.squeeze(z_nominal).tolist())

    z_nominal = z_nominal.reshape(-1, 1)

    bus_i = bus_to_numeric[bus_name]

    # variable
    z = cp.Variable(shape=(num_branches, 1))

    # objective
    minimal_distance = cp.norm(z-z_nominal, 2)

    # add constraints
    G_a_rowi = G_a[bus_i,:].reshape(1,-1)
    B_a_rowi = B_a[bus_i,:].reshape(1,-1)

    Pi_z = G_a_rowi @ cp.sqrt(1-cp.square(z)) + B_a_rowi @ z + d_active[bus_i,0]
    constraints = [ Pi_z <= Pmin[bus_i] ]

    problem = cp.Problem(cp.Minimize(minimal_distance), constraints)

    # Indicate the solver when solving the problem
    solver = cp.CVXOPT  # Replace with the solver of your choice
    problem.solve(solver=solver, ignore_dpp=True)

    if problem.status in ["infeasible", "unbounded"]:
        print(5*' ' +"Model not solved to (sub)optimality using CVXPY!!!")
        print(5*' ' +"Solver status: ", problem.status)
        # sys.exit(0)

        return None

    z_base = z.value
    Pi_z_base = G_a_rowi @ np.sqrt(1-z_base**2) + B_a_rowi @ z_base + d_active[bus_i,0]
    Pi_z_base = Pi_z_base[0,0]
    # print(r'$P_i(z^{base})$:', Pi_z_base)
    # print(r'$P_i^{\min}$:', Pmin[bus_i])

    print('Success!')


    return np.squeeze(z_base).tolist()


In [34]:
# basePoint_of_Pi(3, z_interior)

### Find base points for Qi_min

In [35]:
def basePoint_of_Qi(bus_name, z_nominal):
    print('Linearize Q{}...'.format(bus_name))
    # print('An interior point is ', np.squeeze(z_nominal).tolist())

    z_nominal = z_nominal.reshape(-1, 1)

    bus_i = bus_to_numeric[bus_name]

    # variable
    z = cp.Variable(shape=(num_branches, 1))

    # objective
    minimal_distance = cp.norm(z-z_nominal, 2)

    # add constraints
    B_r_rowi = B_r[bus_i,:].reshape(1,-1)
    G_r_rowi = G_r[bus_i,:].reshape(1,-1)

    Qi_z = B_r_rowi @ cp.sqrt(1-cp.square(z)) + G_r_rowi @ z + d_reactive[bus_i,0]
    Qi_z = Qi_z[0,0]

    constraints = [ Qi_z <= Qmin[bus_i] ]

    problem = cp.Problem(cp.Minimize(minimal_distance), constraints)

    # Indicate the solver when solving the problem
    solver = cp.CVXOPT  # Replace with the solver of your choice
    problem.solve(solver=solver, ignore_dpp=True)

    if problem.status in ["infeasible", "unbounded"]:
        print(5*' ' +"Model not solved to (sub)optimality using CVXPY!!!")
        print(5*' ' +"Solver status: ", problem.status)
        # sys.exit(0)

        return None

    z_base = z.value
    Qi_z_base = B_r_rowi @ np.sqrt(1-z_base**2) + G_r_rowi @ z_base + d_reactive[bus_i,0]
    Qi_z_base = Qi_z_base[0,0]
    # print(r'$Q_i(z^{base})$:', Qi_z_base)
    # print(r'$Q_i^{\min}$:', Qmin[bus_i])
    print('Success!')


    return np.squeeze(z_base).tolist()


In [36]:
# basePoint_of_Qi(2, z_interior)

### Save base points

In [59]:
all_bus_names = bus_df['bus'].to_numpy()
# print('all_bus_names:', all_bus_names)

Pi_basePoints = dict()
Qi_basePoints = dict()

for bus_name in all_bus_names:
    Pi_basePoints[bus_name] = basePoint_of_Pi(bus_name, z_interior)
    Qi_basePoints[bus_name] = basePoint_of_Qi(bus_name, z_interior)

# print('Pi_basePoints:', Pi_basePoints)
# print('Qi_basePoints:', Qi_basePoints)


Linearize P1...
Success!
Linearize Q1...
Success!
Linearize P2...
Success!
Linearize Q2...
Success!
Linearize P3...
Success!
Linearize Q3...
Success!
Linearize P4...
Success!
Linearize Q4...
Success!
Linearize P5...
Success!
Linearize Q5...
Success!
Linearize P6...
Success!
Linearize Q6...
Success!
Linearize P7...
Success!
Linearize Q7...
Success!
Linearize P8...
Success!
Linearize Q8...
Success!
Linearize P9...
Success!
Linearize Q9...
Success!
Linearize P10...
Success!
Linearize Q10...
Success!
Linearize P11...
Success!
Linearize Q11...
Success!
Linearize P12...
Success!
Linearize Q12...
Success!
Linearize P13...
Success!
Linearize Q13...
Success!
Linearize P14...
Success!
Linearize Q14...
Success!
Linearize P15...
Success!
Linearize Q15...
Success!
Linearize P16...
Success!
Linearize Q16...
Success!
Linearize P17...
Success!
Linearize Q17...
Success!
Linearize P18...
Success!
Linearize Q18...
Success!
Linearize P19...
Success!
Linearize Q19...
Success!
Linearize P20...
Success!
Line

# Minimize generation costs

In [45]:
with open(project_path+subproject2 + 'distributed_gencost_data.pkl', 'rb') as file:
    distributed_gencost_data = pickle.load(file)

buscosts = np.array(distributed_gencost_data['cost_coefficients'])
genidx = np.array(distributed_gencost_data['distributed_gen_idx_list'])
gencosts = buscosts[genidx]

genidx = genidx
gencosts = gencosts.reshape(-1,1)

print(genidx.shape)
print(gencosts.shape)

(18,)
(18, 1)


In [39]:
print(np.squeeze(gencosts))
print(np.squeeze(Pmin[genidx]))

[1.02 3.42 5.4  0.4  0.87 1.38 2.4  7.84 7.56 1.56 1.56 2.07 4.96 2.67
 4.85 2.66 1.25 6.51]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [60]:
def min_costs(iteration_number):
    save_path = project_path+subproject2+our_method_path

    p_max = Pmax.reshape(-1, 1)
    p_min = Pmin.reshape(-1, 1)
    q_max = Qmax.reshape(-1, 1)
    q_min = Qmin.reshape(-1, 1)

    ## ------ CVXPY Environment: Define the problem ------ ##
    # variables
    # state = cp.Variable(shape=(2, 1), nonneg=True)
    z = cp.Variable(shape=(num_branches, 1))

    # expressions
    p = cp_p_in_transformed_space(z)# active nodal power
    q = cp_q_in_transformed_space(z)# reactive nodal power
    # p_basePoint = get_p_basePoint()
    # q_basePoint = get_q_basePoint()

    # linearized_p = # linearize active nodal power function
    # linearized_q = # linearize reactive nodal power function

    # objective
    total_gencosts = cp.sum(gencosts.T@p[genidx]) # sum of active nodal power at generators
    # total_losses = cp.sum(p) # total transmission losses

    # constraints
    constraints = [
                    z >= -1,
                    z <= 1,
                    p <= p_max,
                    q <= q_max
    ]
    # add linearized constraints
    for bus_i, bus_name in enumerate(bus_lists):
        # print('bus name is {}, index is {}'.format(bus_name, bus_i))
        base_point = Pi_basePoints[bus_name]
        if base_point is not None:
            z_base = np.array(base_point).reshape(-1,1)
            constraints += [linearize_Pi(bus_i, z_base, z)>=Pmin[bus_i]]

    for bus_i, bus_name in enumerate(bus_lists):
        base_point = Qi_basePoints[bus_name]
        if base_point is not None:
            z_base = np.array(base_point).reshape(-1,1)
            constraints += [linearize_Qi(bus_i, z_base, z)>=Qmin[bus_i]]

    # problem = cp.Problem(cp.Minimize(total_losses), constraints)
    problem = cp.Problem(cp.Minimize(total_gencosts), constraints)

    # problem.solve(ignore_dpp=True)

    # Indicate the solver when solving the problem
    solver = cp.CVXOPT  # Replace with the solver of your choice
    problem.solve(solver=solver, ignore_dpp=True)

    if problem.status in ["infeasible", "unbounded"]:
        print(5*' ' +"Model not solved to (sub)optimality using CVXPY!!!")
        print(5*' ' +"Solver status: ", problem.status)
        sys.exit(0)
    z_solution  = z.value

    # print('Optimal z solution:\n', z_solution)
    # phase_angle_solution = recover_phase_angle(z_solution)
    # # phase_diff_solution = np.arcsin(z_solution)

    optimal_objective_value = total_gencosts.value
    # optimal_objective_value = total_generation.value
    print('Minimal total costs:\n', optimal_objective_value)

    z_solution = z_solution.reshape(-1,1)
    p_solution = np.array([round(x, 3) for x in np.squeeze(p.value)]).reshape(-1,1)
    q_solution = np.array([round(x, 3) for x in np.squeeze(q.value)]).reshape(-1,1)


    min_costs_results = dict()
    min_costs_results['p_solution'] = np.squeeze(p_solution)
    min_costs_results['q_solution'] = np.squeeze(q_solution)
    min_costs_results['z_solution'] = np.squeeze(z_solution)

    # Save the dictionary to the pickle file
    with open(save_path + 'iter' + str(iteration_number) +'_min_costs_results.pkl', 'wb') as file:
        pickle.dump(min_costs_results, file)

    # with open(filename, 'rb') as file:
    #     read_file = pickle.load(file)

    results = np.concatenate([p_max, p_solution, p_min, q_max, q_solution, q_min], axis=1)
    column_names = ['p_max', 'p_solution', 'p_min', 'q_max', 'q_solution', 'q_min']
    results_df = pd.DataFrame(results, columns=column_names)
    print(results_df)

    np.save(project_path+subproject2+'iter' + str(iteration_number+1) +'_z_interior', z_solution)

    return


In [61]:
min_costs(ITER_NUM)

Minimal total costs:
 176.63766736202516
          p_max  p_solution  p_min       q_max  q_solution       q_min
0     27.908664       5.706    0.0   71.990105       0.126    0.020000
1     -0.200000      -0.200  -10.0   10.000000       0.197    0.010000
2      0.040743       0.012    0.0   10.000000       0.000    0.000000
3     -0.400000      -0.400  -10.0   10.000000       0.395    0.020000
4     -0.200000      -0.200  -10.0   10.000000       0.202    0.010000
..          ...         ...    ...         ...         ...         ...
109    0.063596       0.000    0.0   10.000000       0.012   -0.015231
110   -0.200000      -0.200  -10.0   10.000000       0.198    0.010000
111   -0.200000      -0.200  -10.0   10.000000       0.205    0.010000
112   -0.400000      -0.400  -10.0   10.000000       0.400    0.020000
113  200.000000      22.692    0.0  200.000000      -8.019 -200.000000

[114 rows x 6 columns]


# SOCP relaxation

In [42]:
def p_socp(R, I):

    return G_a @ R + B_a @ I + d_active

def q_socp(R, I):

    return B_r @ R + G_r @ I + d_reactive


In [43]:
def socp_relaxation_mincosts():
    save_path = project_path+subproject2+socp_path

    p_max = Pmax.reshape(-1, 1)
    p_min = Pmin.reshape(-1, 1)
    q_max = Qmax.reshape(-1, 1)
    q_min = Qmin.reshape(-1, 1)

    # Define and solve the CVXPY problem.
    R = cp.Variable(shape=(num_branches, 1))
    I = cp.Variable(shape=(num_branches, 1))

    p = p_socp(R, I)# active nodal power
    q = q_socp(R, I)# reactive nodal power

    # objective
    total_gencosts = cp.sum(gencosts.T@p[genidx]) # total transmission losses

    # We use cp.SOC(t, x) to create the SOC constraint ||x||_2 <= t.
    # soc_constraints = [
    #     cp.SOC(c[i].T @ x + d[i], A[i] @ x + b[i]) for i in range(m)
    # ]

    # constraints
    constraints = [
                    R >= 0,
                    p <= p_max,
                    p >= p_min,
                    q <= q_max,
                    q >= q_min,
                    cp.square(R) + cp.square(I) <= 1
    ]

    prob = cp.Problem(cp.Minimize(total_gencosts), constraints)
    prob.solve()

    if prob.status in ["infeasible", "unbounded"]:
        print(5*' ' +"Model not solved to (sub)optimality using CVXPY!!!")
        print(5*' ' +"Solver status: ", prob.status)
        sys.exit(0)

    # Print result.
    print("The optimal value is", prob.value)
    # print("A solution x is")
    # print(x.value)
    # for i in range(m):
    #     print("SOC constraint %i dual variable solution" % i)
    #     print(soc_constraints[i].dual_value)

    R_solution = R.value
    I_solution = I.value
    per_line_slack = R_solution**2 + I_solution**2
    avg_slack = np.mean(R_solution**2 + I_solution**2 - 1)
    print('avg_slack:', avg_slack)
    print('per line:', np.squeeze(per_line_slack).tolist())

    threshold = 0.98
    for i, each in enumerate(np.squeeze(per_line_slack)):
        if each <= threshold:
            print('{}-th line has slack {}'.format(i, each))

In [44]:
socp_relaxation_mincosts()

The optimal value is 39.970768104196814
avg_slack: -0.0008423686286453938
per line: [0.9999998127438893, 0.9999998744526433, 0.995597619928564, 0.9999996517860547, 0.9999997918385912, 0.9999997180771705, 0.9999571929769911, 0.9999998614090676, 0.9999998630003784, 0.9495259317399971, 0.9999999707140846, 0.9999999597576735, 0.9999996963639405, 0.9999998865365629, 0.9999999066709729, 0.9999999445970097, 1.0000023202695216, 1.000002270819309, 0.9999998122379494, 0.9999998965672838, 0.9999999225723863, 0.9999998148682094, 0.9999999051102005, 0.9989438711602359, 0.999999937139968, 0.9999456490878921, 0.9999284787818752, 0.999999772729304, 0.9998261771351501, 0.9999998451442373, 0.9999998917381014, 0.9999998342236217, 0.9999998704273866, 0.999999756004117, 1.0000080675680865, 0.9999897784256357, 0.9999998061731847, 0.999999783103476, 0.9999997435114155, 0.9999998910094253, 0.9999998391498289, 0.9999998159893372, 0.9999998973190881, 0.9997070973473245, 0.9999998218120127, 0.9996822742237682, 0